In [2]:
%reload_ext autoreload
%autoreload 2
import tensorflow as tf 
from tensorflow import keras 
import numpy as np 
import convo as c 
import convL 
import model 
import modularized as m 
import batching
import poolingL
import gin

C:\Users\dnepr\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import sys
sys.path.append("/content/drive/Othercomputers/Mein PC/Completed")

In [3]:

def convolution(X, ref_A,  ref_B, mask=None):

    #X_a = tf.gather(X, ref_A, axis=0)
    X_b = tf.gather(X, ref_B, axis=0)
    if mask is not None:
        X_b *= mask
    X_aggregate = tf.scatter_nd(tf.expand_dims(
        ref_A, axis=- 1), X_b, shape=tf.shape(X))

    return X_aggregate

# python efficiency -->apply twice in layer  corresponds to D^(-1/2) X


def normalization(X, ref_A, ref_B, mask=None):
    X_norm = tf.ones(shape=(tf.shape(X)[0], tf.shape(
        mask)[1] if mask is not None else 1))  # num_nodes
    d = convolution(X_norm, ref_A, ref_B, mask) + 1
    d_rsqrt = tf.math.rsqrt(d)


inputs = {'X': keras.Input(shape=(9,), dtype=tf.float32, name='X'),
          'ref_A': keras.Input((), dtype=tf.int32, name='ref_A'),
          'ref_B': keras.Input((), dtype=tf.int32, name='ref_B'),
          'num_nodes': keras.Input((), dtype=tf.int32, name='num_nodes')}


In [4]:
conv23 = convL.ConvolutionLayer(9)(inputs) 
conv2_layer = convL.ConvolutionLayer(9,  activation='relu', drop_type= None, p=0.5)
conv2_output = conv2_layer(inputs)


In [5]:
# testinstance X --> outputs
my_test_dict = {'X': tf.constant([[12, 10, 3, 4, 8, 23, 8, 9, 10], [12, 10, 3, 4, 8, 23, 8, 9, 10], [12, 10, 3, 4, 8, 23, 8, 9, 10], [12, 10, 3, 4, 8, 23, 8, 9, 10],
 [12, 10, 3, 4, 8, 23, 8, 9, 10], [12, 10, 3, 4, 8, 23, 8, 9, 10], [12, 10, 3, 4, 8, 23, 8, 9, 10], [12, 10, 3, 4, 8, 23, 8, 9, 10], [12, 10, 3, 4, 8, 23, 8, 9, 10]],
  dtype=tf.float32, name='X'), 'ref_A': tf.constant([
    0, 1, 2, 2, 3, 3, 2, 2], dtype=tf.int32, name = 'ref_A'), 'ref_B': tf.constant([5, 2, 5, 3, 5, 4, 4, 6], dtype = tf.int32, name = 'ref_B'), 'num_nodes': tf.constant(7, dtype = tf.int32)}

conv2_layer.w # instance tf.var 
conv2_layer.w.assign(tf.ones(tf.shape(conv2_layer.w), dtype = tf.dtypes.float32))

print(conv2_layer.w, conv2_layer.b)
conv2_out  =conv2_layer(my_test_dict)
print(conv2_out)

#tf.print(conv2_out)

# tf.shape(ref_A) -->  (8,) ??? 


<tf.Variable 'convolution_layer_4/Variable:0' shape=(9, 9) dtype=float32, numpy=
array([[1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.]], dtype=float32)> <tf.Variable 'convolution_layer_4/Variable:0' shape=(9,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>
{'X': <tf.Tensor: shape=(9, 9), dtype=float32, numpy=
array([[ 87.     ,  87.     ,  87.     ,  87.     ,  87.     ,  87.     ,
         87.     ,  87.     ,  87.     ],
       [130.5    , 130.5    , 130.5    , 130.5    , 130.5    , 130.5    ,
        130.5    , 130.5    , 130.5    ],
       [117.75913, 117.75913, 117.75913, 117.75913, 117.75913, 117.75913,

In [6]:
# DropOut

conv3_dropout = convL.ConvolutionLayer(9, activation='relu', drop_type= 'DropOut', p = 0.7)
conv3_dropout(conv2_out)


{'X': <tf.Tensor: shape=(9, 9), dtype=float32, numpy=
 array([[7.8445328e+01, 0.0000000e+00, 0.0000000e+00, 8.5335808e+01,
         3.4294424e+00, 5.8793648e+01, 5.2916710e+01, 0.0000000e+00,
         7.1703979e+01],
        [8.6480362e+01, 0.0000000e+00, 3.8319908e+01, 6.0256466e+01,
         0.0000000e+00, 7.4317863e+01, 1.6147884e+01, 8.7625465e+00,
         2.1899755e+01],
        [2.5787003e+01, 0.0000000e+00, 0.0000000e+00, 3.6035595e+01,
         0.0000000e+00, 6.4851303e+01, 0.0000000e+00, 3.5905659e+01,
         0.0000000e+00],
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 3.3872981e+00,
         0.0000000e+00, 7.0190872e+01, 0.0000000e+00, 1.1689650e+01,
         0.0000000e+00],
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
         0.0000000e+00],
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 7.2469387e+00,
         2.5865694e+01, 0.0000000e+00, 0.0000000e+00, 2.574134

In [7]:
# NodeSampling 

conv3nodesampl = convL.ConvolutionLayer(9, None, 'NodeSampling', p = 0.9)
conv3nodesampl(conv2_out)


# has to remove row-wise but does not !!! 



{'X': <tf.Tensor: shape=(9, 9), dtype=float32, numpy=
 array([[  14.306793,   37.423107,   35.26494 , -179.94464 ,  -37.427517,
          -88.062225,  -35.209698,  181.91411 ,    6.850677],
        [  21.460188,   56.13466 ,   52.89742 , -269.91696 ,  -56.141277,
         -132.09334 ,  -52.814545,  272.8712  ,   10.27602 ],
        [  19.365007,   50.654163,   47.73297 , -243.56462 ,  -50.66013 ,
         -119.196915,  -47.6582  ,  246.23045 ,    9.272761],
        [  14.306793,   37.423107,   35.26494 , -179.94464 ,  -37.427517,
          -88.062225,  -35.209698,  181.91411 ,    6.850677],
        [  14.306793,   37.423107,   35.26494 , -179.94464 ,  -37.427517,
          -88.062225,  -35.209698,  181.91411 ,    6.850677],
        [  14.306793,   37.423107,   35.26494 , -179.94464 ,  -37.427517,
          -88.062225,  -35.209698,  181.91411 ,    6.850677],
        [  14.306793,   37.423107,   35.264935, -179.94461 ,  -37.427513,
          -88.06223 ,  -35.209698,  181.91411 ,    6.850

In [8]:
# DropEdge 

conv3_dropedge = convL.ConvolutionLayer(9, None, 'DropEdge', p = 0.7)
conv3_dropedge(conv2_out)


{'X': <tf.Tensor: shape=(9, 9), dtype=float32, numpy=
 array([[  99.001465,  -17.542274,   56.330723,  -16.252144,   21.224442,
          -23.462202,  -36.39919 ,  -41.064137,   39.332222],
        [ 282.50595 ,  -50.05777 ,  160.7427  ,  -46.376343,   60.56508 ,
          -66.95069 , -103.867035, -117.17869 ,  112.23662 ],
        [ 233.00523 ,  -41.286655,  132.57735 ,  -38.250282,   49.952854,
          -55.219555,  -85.667435,  -96.64664 ,   92.570496],
        [  99.001465,  -17.542274,   56.330723,  -16.252144,   21.224442,
          -23.462202,  -36.39919 ,  -41.064137,   39.332222],
        [  99.001465,  -17.542274,   56.330723,  -16.252144,   21.224442,
          -23.462202,  -36.39919 ,  -41.064137,   39.332222],
        [  99.001465,  -17.542274,   56.330723,  -16.252144,   21.224442,
          -23.462202,  -36.39919 ,  -41.064137,   39.332222],
        [  99.00147 ,  -17.542263,   56.330723,  -16.25214 ,   21.224447,
          -23.46221 ,  -36.39919 ,  -41.064137,   39.332

In [9]:
# DropOut 

start = my_test_dict
conv1 = convL.ConvolutionLayer(9, 'relu', 'GDC', 0.4)
conv1_out = conv1(start)


print(conv1.w, conv1.b) # random 


print(conv1_out)



<tf.Variable 'convolution_layer_8/Variable:0' shape=(9, 9) dtype=float32, numpy=
array([[-0.37736   , -0.2948938 ,  0.46817565,  0.38705665,  0.5750669 ,
        -0.31486434, -0.32494769, -0.21939069,  0.56871843],
       [-0.26783437, -0.25802672,  0.20348537, -0.55369204,  0.56537807,
         0.38116795, -0.2259967 ,  0.09634995, -0.0572089 ],
       [-0.22219107, -0.56085074, -0.04278153,  0.2323026 ,  0.38344967,
         0.3166154 ,  0.47032273,  0.5371926 ,  0.08849227],
       [-0.19414714, -0.16666967, -0.07452911,  0.08353144, -0.5361067 ,
         0.5446013 , -0.16747051,  0.28651482, -0.0316062 ],
       [ 0.31165004, -0.25775224,  0.4489411 ,  0.29564232,  0.34509647,
        -0.21201673, -0.25880486,  0.41813785,  0.01325881],
       [-0.17241466,  0.44429302,  0.22546649, -0.49706393, -0.53425586,
        -0.05679011, -0.00399089,  0.05899215,  0.21156919],
       [-0.53552955, -0.17039299, -0.47822803,  0.33396602, -0.17504337,
         0.12606835,  0.46456492, -0.34909

In [18]:
# 

inputs = model.inputs 
first_model = m.create_model(['Convolution', 'Pooling', 'Dense'], inputs)

second_model = m.create_model(['Convolution', 'Pooling'], inputs)

c1 = convL.ConvolutionLayer(32)(inputs)
p1 = poolingL.Pooling()(c1)
mip = gin.GConvoLayer(32)(p1)

TypeError: tuple indices must be integers or slices, not str

In [17]:
привет = 1 
 
m1 = tf.constant(np.arange(12)) 
tf.reshape(m1,(3,4))
m2 = tf.constant([[2,1,2,4],[8, 3, 1,4],[6,1,2,3]]) 
m2.shape

def vector_length(array1): 
    return np.sqrt(np.sum(array1 ** 2, axis = -1))

def matrix_product(X):
    return np.matmul(X, X.T) 

def cosine(X): 
    return 1- matrix_product(X/vector_length(X).reshape((-1,1)))


def ad(X): 
    return np.mean(cosine(X))
